In [26]:
import json


with open('json/balanced_track_with_genre.json', "r") as f:
    
    song_list = json.load(f)
    
len(song_list)

50140

In [28]:
song_set = set([item['track_id'] for item in song_list])

len(song_set.intersection(failed))

398

In [10]:
import lmdb
from tqdm.notebook import tqdm

env = lmdb.open("data/MSD/MSD_ID_to_log_mel_spectrogram.lmdb", 
                  subdir=False, readonly=True, lock=False,
                  readahead=False, meminit=False, max_readers=1)           

count = 0

pbar = tqdm(song_list)
for item in pbar:
    
    with env.begin(write=False) as txn:
        data = txn.get(item["track_id"][:17].encode())
        
    if data is None:
        count += 1
        print(item['track_id'])

    pbar.set_description("Current Count: %d" %count)


TRMYZDQ128F42B4D85
TRHJRMH128F42B4D82
TRHJERG128F4271225
TRBMOQG128F4271219
TRQODRD128F427121C
TRZOIGH128F427121F
TRZXDHU128F4271230
TROWKSJ128F931A167
TRGYUKP128F9316385
TRBAXDD12903CCFC62
TRZWWZU128F9353DDA
TRIOIFD128F9353D97
TRFJONP128F92D5C30
TRFOIDN128F146B8C6
TRZGUNN128E07862F7
TRMPQGC12903CD1215
TRCZXXQ128F932F48B
TRGDXJR128F92D5619
TRIVCBE128E07862E3
TRLSPEP128F92F49A8
TRJAJTP128F92F49A5
TRMCOJO128F9330F7C
TRIAJNC128F92F7E77
TRHNCIR128F42334A5
TRCHMBU128F423457B
TRQECHB128F4234580
TRZRONZ128F9301D2B
TRIVUXL128F9311BEC
TRSHSIJ128F9301D89
TRQOPRR12903CED6A3
TRZRDFC128F428CD99
TRVQSFC128F428CDB2
TRPUYOR128F9324E9A
TRCEIKR128F424A017
TRWDPUY128F9339D39
TRVLIPF128F9336661
TRWDLRG12903CC5701
TRMSHQN128E0792879
TRHEVPM128F145482A
TRPBUFT128F145525C
TREWTIO128F145497E
TRCZOYM128F9314AE3
TRUUEWG128F9300996
TRRTHPH128F930767B
TRCAUVH12903CCAB60
TRRSQZX12903CCAB16
TRIRDXG12903CED07D
TRNLUNF12903CCAB29
TREQLHP12903CCAB6E
TRLRGVX128E078EC1B
TRMEKFK128F931C778
TRHZAXL128F931C76F
TRHSXPZ128F9

In [26]:
failed = set()

with open("data/msd_genre/failed.csv", "r") as f:
    
    for row in f.readlines():
        failed.add(row.strip())
    
len(failed)

398

In [30]:

with open("data/msd_genre/msd_id_to_genre.csv", "w") as f:
    
    f.write("msd_id, genre \n")
    
    for song in song_list:
        if song['track_id'] not in failed:
            f.write(song['track_id'] + ", " + song['genre'] + "\n")


with open("data/msd_genre/msd_id_to_genre.csv", "r") as f:
    
    print(len(f.readlines()))  

49729


In [ ]:
%load_ext autoreload
%autoreload 2

In [1]:
import torch
from source.datasets.fast_datasets import *
from source.models_base.mb_vggish import MusicBertVGGish

BERT = MusicBertVGGish(name="test").cuda()
# BERT.load_pretrained()
BERT.load_model("models/music_bert_mlm_increased_masking.pth")
BERT.eval()

genre_dataset = GTZANFastDataset()

In [3]:
%matplotlib widget

from source.utils.plot_utils import visualize_vectors

visualize_vectors(BERT, genre_dataset)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

0.12964496
0.12491662
0.09233757
0.08564526
0.07158065
0.053951513
0.042955186
0.03548927
0.02420029
0.018629117
0.013991229
0.010240084
0.0046078973


In [5]:
%matplotlib widget

import matplotlib.pyplot as plt

values = [0.12964496,
          0.12491662,
          0.09233757,
          0.08564526,
          0.07158065,
          0.053951513,
          0.042955186,
          0.03548927,
          0.02420029,
          0.018629117,
          0.013991229,
          0.010240084,
          0.0046078973]

plt.plot(values)
plt.ylabel("Std Dev across Time")
plt.xlabel('Layers')
plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [84]:
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np
from transformers import BertTokenizer
import torch

from source.utils.plot_utils import plot_layers

x = torch.rand((31,1,768)).cuda()

print(torch.mean(x), torch.std(x))

output, out_layers = BERT.BERT.encoder(x, return_layers_out=True)
    
plot_layers(out_layers, graph_height=3)

tensor(0.5015, device='cuda:0') tensor(0.2889, device='cuda:0')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [86]:
%matplotlib widget
import matplotlib.pyplot as plt

sample_sound = torch.tensor(genre_dataset[0]['song_features']).to(BERT.get_device()).unsqueeze(1)

output, extras = BERT(sample_sound, return_extras=True, add_special_tokens=False)

vggish_features = extras['encoded_sound_1'].cpu().squeeze().numpy()

print(np.mean(vggish_features), np.std(vggish_features))

vggish_mean = np.mean(vggish_features, axis = 0)
vggish_features -= vggish_mean

print(vggish_mean.shape)


plot_layers([vggish_features, np.expand_dims(vggish_mean,0)], graph_height=4)

# #fig, axs = plt.subplots(1, 1, figsize=(10, 10))
# fig = plt.figure(figsize=(10,4))
# ax = fig.add_subplot(111)
# cax = ax.matshow(vggish_features)#, vmin=0, vmax=1)
# fig.colorbar(cax, orientation="horizontal")

# fig = plt.figure(figsize=(10,4))
# ax = fig.add_subplot(211)
# cax = ax.matshow()#, vmin=0, vmax=1)
# fig.colorbar(cax, orientation="horizontal")


np.std(vggish_features*10, axis=0), 

0.52116346 0.270439
(128,)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(array([0.26265582, 0.24314113, 0.4904722 , 0.6371486 , 0.37582925,
        0.42569324, 0.66943693, 0.8077324 , 1.0571753 , 0.37625763,
        0.6733464 , 0.91625446, 1.100779  , 1.3601087 , 1.1524223 ,
        1.1396981 , 1.0458912 , 0.93207526, 1.0422192 , 1.296333  ,
        0.87290096, 0.98499715, 0.8561328 , 0.90536267, 0.9290474 ,
        1.3575609 , 0.8282712 , 1.3497044 , 0.644908  , 1.2908901 ,
        0.9657755 , 1.4762956 , 1.0869035 , 1.6353174 , 1.3196028 ,
        0.84952545, 1.9567202 , 1.3271974 , 1.0563889 , 1.0341446 ,
        1.5252723 , 1.0571796 , 1.3545461 , 1.734576  , 1.1667393 ,
        0.83480144, 1.3493408 , 1.2828995 , 1.6204212 , 1.2651197 ,
        1.299198  , 1.3764689 , 1.7764692 , 1.4029831 , 1.4645967 ,
        1.7184514 , 1.3595887 , 1.4450147 , 1.1203052 , 1.7753474 ,
        1.2803429 , 1.3585453 , 1.8191541 , 1.5642653 , 1.7465247 ,
        1.6197777 , 1.394955  , 1.443027  , 1.5189043 , 1.6627133 ,
        1.8216153 , 1.3241923 , 0.8132296 , 1.67

In [49]:
type(torch.tensor([])) == torch.Tensor

True

In [4]:
import csv
from tqdm import tqdm

ids = set()

labels_count = {}

reader = csv.reader(open('data/MTAT/annotations_final.csv', 'r'), delimiter='\t')
classes = next(reader, None)[1:-1]

for label in classes:
    labels_count[label] = 0

for row in reader:
    for label, value in zip(classes, row[1:-1]):
        labels_count[label] += int(value)
    
    
    
len(labels_count)

188

In [8]:
%matplotlib widget
import matplotlib.pyplot as plt

labels = [k for k in labels_count.keys()]
values = [v for _,v in labels_count.items()]

plt.barh(labels[:10], values[:10])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<BarContainer object of 10 artists>

In [14]:
top50tags_count = sorted(labels_count.items(), key=lambda item: item[1], reverse=True)[:50]
top50tags = [k for k,v in top50tags_count]

for i, cl in enumerate(classes):
    if cl in top50tags:
        print (i, end=", ")

0, 7, 10, 13, 14, 18, 27, 28, 34, 35, 39, 42, 45, 46, 47, 55, 59, 63, 67, 68, 70, 71, 78, 80, 81, 82, 84, 90, 91, 97, 115, 120, 123, 125, 132, 143, 144, 146, 147, 151, 153, 154, 159, 160, 165, 169, 173, 175, 177, 180, 

In [4]:
import torch

x = torch.rand(2,4)

y = torch.softmax(x.unsqueeze(1).repeat(1,3,1), 1)

y.shape, torch.diagonal(y).shape

(torch.Size([2, 3, 4]), torch.Size([4, 2]))

In [5]:
import torch.nn as nn

rnn = nn.RNN(10,15, 2)

x, h = rnn(torch.randn(4, 1, 10))

x.shape, h.shape

(torch.Size([4, 1, 15]), torch.Size([2, 1, 15]))

In [2]:
from source.utils.generic_utils import allDone

allDone()